# LSTM con embeddings + double translated

Modelo básico con los embeddings de fasttext

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import csv
import numpy as np
import tensorflow as tf
import random

np.random.seed(2019)
tf.random.set_random_seed(2019)
random.seed(2019)

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_test = pd.read_table("../../../data/es/reference_es.tsv", header=None, 
                        names=["text", "HS", "TR", "AG"], quoting=csv.QUOTE_NONE)


text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]
text_test, y_test = df_test["text"], df_test["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))
print("Instancias de test: {}".format(len(df_test)))

Instancias de entrenamiento: 4500
Instancias de desarrollo: 500
Instancias de test: 1600


In [2]:
train_synth_en = pd.read_table("../../../data/es/train_es.synth.en.tsv", index_col="id", quoting=csv.QUOTE_NONE)
train_synth_fr = pd.read_table("../../../data/es/train_es.synth.fr.tsv", index_col="id", quoting=csv.QUOTE_NONE)

text_train_synth_en, y_train_synth_en = train_synth_en["text"], train_synth_en["HS"]
text_train_synth_fr, y_train_synth_fr = train_synth_fr["text"], train_synth_fr["HS"]

Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 200000

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_dev = tokenizer.texts_to_sequences(text_dev)
X_test = tokenizer.texts_to_sequences(text_test)
X_train_synth_en = tokenizer.texts_to_sequences(text_train_synth_en)
X_train_synth_fr = tokenizer.texts_to_sequences(text_train_synth_fr)

max_length = 30

X_train = pad_sequences(X_train, max_length)
X_dev = pad_sequences(X_dev, max_length)
X_test = pad_sequences(X_test, max_length)
X_train_synth_en = pad_sequences(X_train_synth_en, max_length)
X_train_synth_fr = pad_sequences(X_train_synth_fr, max_length)

Using TensorFlow backend.


Carguemos embeddings

In [4]:
import os
from glob import glob

path_to_embeddings = os.path.expanduser("~/WordVectors/")

print("Available embeddings: ", glob(os.path.join(path_to_embeddings, "*.vec")))

Available embeddings:  ['/home/jmperez/WordVectors/UBA_w5_200.vec', '/home/jmperez/WordVectors/wiki.es.vec', '/home/jmperez/WordVectors/UBA_w5_300.vec']


# Twitter Embeddings

In [5]:
import numpy as np

word_to_vec = {}

with open(os.path.join(path_to_embeddings, "UBA_w5_300.vec")) as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            vec = np.asarray(values[1:], dtype="float32")
        except:
            print(("*" * 80  + "\n")*3)
            print("Problema con la sig línea:")
            print(values[:10])
            word = values[1]
            vec = np.asarray(values[2:], dtype="float32")
        word_to_vec[word] = vec
        
embedding_size = len(word_to_vec["hola"])

********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '.', '-0.22232', '0.0052569', '0.47066', '0.13836', '0.15991', '0.19504', '0.00067885', '0.020299']
********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '...', '-0.11666', '-0.083768', '0.028919', '0.29973', '0.21017', '0.27808', '0.063251', '0.090223']
********************************************************************************
********************************************************************************
********************************************************************************

P

In [6]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    embedding_vector = word_to_vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [7]:
embedding_matrix.shape

(200000, 300)

# Sin Synths


In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import CuDNNLSTM, Dropout, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import Adam

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(CuDNNLSTM(100, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 2s 543us/step - loss: 0.6361 - acc: 0.6531 - val_loss: 0.5991 - val_acc: 0.6780
Epoch 2/20
4500/4500 [==============================] - 1s 279us/step - loss: 0.5516 - acc: 0.7267 - val_loss: 0.5550 - val_acc: 0.7120
Epoch 3/20
4500/4500 [==============================] - 1s 283us/step - loss: 0.5114 - acc: 0.7531 - val_loss: 0.5430 - val_acc: 0.7380
Epoch 4/20
4500/4500 [==============================] - 1s 283us/step - loss: 0.4908 - acc: 0.7669 - val_loss: 0.5342 - val_acc: 0.7560
Epoch 5/20
4500/4500 [==============================] - 1s 283us/step - loss: 0.4714 - acc: 0.7809 - val_loss: 0.5358 - val_acc: 0.7520
Epoch 6/20
4500/4500 [==============================] - 1s 286us/step - loss: 0.4552 - acc: 0.7887 - val_loss: 0.5330 - val_acc: 0.7380
Epoch 7/20
4500/4500 [==============================] - 1s 283us/step - loss: 0.4425 - acc: 0.8031 - val_loss: 0.5412 - val_acc: 0.7360
E

In [9]:
from hate.utils import print_evaluation

print("Evaluación sobre dev")
print_evaluation(model, X_dev, y_dev)
print("\n\nEvaluación sobre test")
print_evaluation(model, X_test, y_test)



Evaluación sobre dev
500/500 [==============================] - 0s 98us/step
Loss           : 0.5440
Accuracy       : 0.7760
Precision(1)   : 0.7273
Precision(1)   : 0.8217
Precision(avg) : 0.7745

Recall(1)      : 0.7928
Recall(0)      : 0.7626
Recall(avg)    : 0.7777

F1(1)          : 0.7586
F1(0)          : 0.7910
F1(avg)        : 0.7748


Evaluación sobre test
1600/1600 [==============================] - 0s 107us/step
Loss           : 0.6801
Accuracy       : 0.6875
Precision(1)   : 0.5973
Precision(1)   : 0.7828
Precision(avg) : 0.6900

Recall(1)      : 0.7439
Recall(0)      : 0.6479
Recall(avg)    : 0.6959

F1(1)          : 0.6626
F1(0)          : 0.7090
F1(avg)        : 0.6858


## Usando los synths generados de entrenamiento (sólo inglés)

In [10]:

X_tr = np.vstack((X_train, X_train_synth_en))
y_tr = np.vstack((y_train.values.reshape(-1, 1), 
                  y_train_synth_en.values.reshape(-1, 1),
))

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(CuDNNLSTM(100))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
model.fit(X_tr, y_tr, 
          validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 5991 samples, validate on 500 samples
Epoch 1/20
5991/5991 [==============================] - 2s 342us/step - loss: 0.6181 - acc: 0.6740 - val_loss: 0.6634 - val_acc: 0.6440
Epoch 2/20
5991/5991 [==============================] - 2s 289us/step - loss: 0.5347 - acc: 0.7518 - val_loss: 0.5452 - val_acc: 0.7320
Epoch 3/20
5991/5991 [==============================] - 2s 279us/step - loss: 0.5043 - acc: 0.7640 - val_loss: 0.5359 - val_acc: 0.7420
Epoch 4/20
5991/5991 [==============================] - 2s 280us/step - loss: 0.4863 - acc: 0.7748 - val_loss: 0.5315 - val_acc: 0.7400
Epoch 5/20
5991/5991 [==============================] - 2s 285us/step - loss: 0.4791 - acc: 0.7875 - val_loss: 0.5253 - val_acc: 0.7460
Epoch 6/20
5991/5991 [==============================] - 2s 280us/step - loss: 0.4589 - acc: 0.7904 - val_loss: 0.5255 - val_acc: 0.7680
Epoch 7/20
5991/5991 [==============================] - 2s 283us/step - loss: 0.4490 - acc: 0.7997 - val_loss: 0.5441 - val_acc: 0.7320
E

In [12]:
from hate.utils import print_evaluation

print("Evaluación sobre dev")
print_evaluation(model, X_dev, y_dev)
print("\n\nEvaluación sobre test")
print_evaluation(model, X_test, y_test)



Evaluación sobre dev
500/500 [==============================] - 0s 104us/step
Loss           : 0.5189
Accuracy       : 0.7700
Precision(1)   : 0.7257
Precision(1)   : 0.8099
Precision(avg) : 0.7678

Recall(1)      : 0.7748
Recall(0)      : 0.7662
Recall(avg)    : 0.7705

F1(1)          : 0.7495
F1(0)          : 0.7874
F1(avg)        : 0.7684


Evaluación sobre test
1600/1600 [==============================] - 0s 108us/step
Loss           : 0.6262
Accuracy       : 0.6863
Precision(1)   : 0.6031
Precision(1)   : 0.7626
Precision(avg) : 0.6829

Recall(1)      : 0.7000
Recall(0)      : 0.6766
Recall(avg)    : 0.6883

F1(1)          : 0.6480
F1(0)          : 0.7170
F1(avg)        : 0.6825


## Usando Inglés + Francés

In [13]:

X_tr = np.vstack((X_train, X_train_synth_en, X_train_synth_fr))
y_tr = np.vstack((
    y_train.values.reshape(-1, 1), 
    y_train_synth_en.values.reshape(-1, 1),
    y_train_synth_fr.values.reshape(-1, 1)
))

In [14]:

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(CuDNNLSTM(100))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
model.fit(X_tr, y_tr, 
          validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 7168 samples, validate on 500 samples
Epoch 1/20
7168/7168 [==============================] - 3s 362us/step - loss: 0.6036 - acc: 0.6770 - val_loss: 0.5610 - val_acc: 0.7160
Epoch 2/20
7168/7168 [==============================] - 2s 278us/step - loss: 0.5290 - acc: 0.7480 - val_loss: 0.5332 - val_acc: 0.7320
Epoch 3/20
7168/7168 [==============================] - 2s 284us/step - loss: 0.5017 - acc: 0.7619 - val_loss: 0.5229 - val_acc: 0.7400
Epoch 4/20
7168/7168 [==============================] - 2s 280us/step - loss: 0.4846 - acc: 0.7737 - val_loss: 0.5207 - val_acc: 0.7580
Epoch 5/20
7168/7168 [==============================] - 2s 280us/step - loss: 0.4703 - acc: 0.7822 - val_loss: 0.5139 - val_acc: 0.7640
Epoch 6/20
7168/7168 [==============================] - 2s 280us/step - loss: 0.4563 - acc: 0.7884 - val_loss: 0.5088 - val_acc: 0.7660
Epoch 7/20
7168/7168 [==============================] - 2s 279us/step - loss: 0.4451 - acc: 0.7976 - val_loss: 0.5267 - val_acc: 0.7480
E

In [15]:
from hate.utils import print_evaluation

print("Evaluación sobre dev")
print_evaluation(model, X_dev, y_dev)
print("\n\nEvaluación sobre test")
print_evaluation(model, X_test, y_test)

Evaluación sobre dev
500/500 [==============================] - 0s 109us/step
Loss           : 0.5086
Accuracy       : 0.7760
Precision(1)   : 0.7594
Precision(1)   : 0.7882
Precision(avg) : 0.7738

Recall(1)      : 0.7252
Recall(0)      : 0.8165
Recall(avg)    : 0.7709

F1(1)          : 0.7419
F1(0)          : 0.8021
F1(avg)        : 0.7720


Evaluación sobre test
1600/1600 [==============================] - 0s 106us/step
Loss           : 0.6264
Accuracy       : 0.6963
Precision(1)   : 0.6192
Precision(1)   : 0.7609
Precision(avg) : 0.6900

Recall(1)      : 0.6848
Recall(0)      : 0.7043
Recall(avg)    : 0.6946

F1(1)          : 0.6504
F1(0)          : 0.7315
F1(avg)        : 0.6909
